In [4]:
# Cell 1: Import required libraries
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import cv2
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tqdm import tqdm

from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, TensorBoard, ModelCheckpoint
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras import layers, models

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/indian-medicinal-leaves-dataset/Indian Medicinal Leaves Image Datasets/Medicinal plant dataset/Nooni/2863.jpg
/kaggle/input/indian-medicinal-leaves-dataset/Indian Medicinal Leaves Image Datasets/Medicinal plant dataset/Nooni/187.jpg
/kaggle/input/indian-medicinal-leaves-dataset/Indian Medicinal Leaves Image Datasets/Medicinal plant dataset/Nooni/2844.jpg
/kaggle/input/indian-medicinal-leaves-dataset/Indian Medicinal Leaves Image Datasets/Medicinal plant dataset/Nooni/708.jpg
/kaggle/input/indian-medicinal-leaves-dataset/Indian Medicinal Leaves Image Datasets/Medicinal plant dataset/Nooni/2866.jpg
/kaggle/input/indian-medicinal-leaves-dataset/Indian Medicinal Leaves Image Datasets/Medicinal plant dataset/Nooni/185.jpg
/kaggle/input/indian-medicinal-leaves-dataset/Indian Medicinal Leaves Image Datasets/Medicinal plant dataset/Nooni/703.jpg
/kaggle/input/indian-medicinal-leaves-dataset/Indian Medicinal Leaves Image Datasets/Medicinal plant dataset/Nooni/2852.jpg
/kaggle/inpu

In [7]:


path = '/kaggle/input/indian-medicinal-leaves-dataset/Indian Medicinal Leaves Image Datasets/Medicinal plant dataset'
no_epochs = 100

In [8]:
# Cell 3: Get the labels
labels = []
for lab in tqdm(os.listdir(path)):
    labels.append(lab)
print(f'No of labels found = {len(labels)}')
no_classes = len(labels)

100%|██████████| 40/40 [00:00<00:00, 377016.09it/s]

No of labels found = 40


In [32]:
print(labels)


['Nooni', 'Nithyapushpa', 'Basale', 'Pomegranate', 'Honge', 'Lemon_grass', 'Mint', 'Betel_Nut', 'Nagadali', 'Curry_Leaf', 'Jasmine', 'Castor', 'Sapota', 'Neem', 'Ashoka', 'Brahmi', 'Amruta_Balli', 'Pappaya', 'Pepper', 'Wood_sorel', 'Gauva', 'Hibiscus', 'Ashwagandha', 'Aloevera', 'Raktachandini', 'Insulin', 'Bamboo', 'Amla', 'Arali', 'Geranium', 'Avacado', 'Lemon', 'Ekka', 'Betel', 'Henna', 'Doddapatre', 'Rose', 'Mango', 'Tulasi', 'Ganike']


In [10]:
# Cell 4: Load images, resize, and normalize
X_train = []
y_train = []
img_per_lab = []
image_size = 224  # Image size

# Define the device (GPU/CPU)
device = "/GPU:0" if tf.config.list_physical_devices('GPU') else "/CPU:0"
print(device)

# Load and preprocess images
for i in labels:
    folderPath = path + '/' + i
    print(f'{labels.index(i)}: {folderPath}')
    imglist = os.listdir(folderPath)
    img_per_lab.append(len(imglist))
    for j in tqdm(imglist):
        img = cv2.imread(folderPath + '/' + j)
        img = cv2.resize(img, (image_size, image_size))
        img = img / 255.0  # Normalize the pixels value  to [0, 1]
        X_train.append(img)
        y_train.append(i)

/GPU:0
0: /kaggle/input/indian-medicinal-leaves-dataset/Indian Medicinal Leaves Image Datasets/Medicinal plant dataset/Nooni


100%|██████████| 146/146 [00:01<00:00, 76.39it/s]


1: /kaggle/input/indian-medicinal-leaves-dataset/Indian Medicinal Leaves Image Datasets/Medicinal plant dataset/Nithyapushpa


100%|██████████| 146/146 [00:01<00:00, 74.95it/s]


2: /kaggle/input/indian-medicinal-leaves-dataset/Indian Medicinal Leaves Image Datasets/Medicinal plant dataset/Basale


...

100%|██████████| 146/146 [00:01<00:00, 77.59it/s]


38: /kaggle/input/indian-medicinal-leaves-dataset/Indian Medicinal Leaves Image Datasets/Medicinal plant dataset/Tulasi


100%|██████████| 146/146 [00:01<00:00, 78.64it/s]


39: /kaggle/input/indian-medicinal-leaves-dataset/Indian Medicinal Leaves Image Datasets/Medicinal plant dataset/Ganike


100%|██████████| 115/115 [00:01<00:00, 74.36it/s]


In [11]:
# Cell 5: Convert to NumPy arrays
X_train = np.array(X_train)
y_train = np.array(y_train)

In [12]:
# Cell 6: Shuffle the dataset
X_train, y_train = shuffle(X_train, y_train, random_state=101)

In [13]:
# Cell 7: Check shape of the training data
print(X_train.shape)  # (num_images, 224, 224, 3)

(5945, 224, 224, 3)


In [14]:
# Cell 8: Split data into training and validation sets (80-20 split)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=101)

In [15]:
# Cell 9: Convert labels to one-hot encoding
y_train_new = [labels.index(i) for i in y_train]
y_train = tf.keras.utils.to_categorical(y_train_new, num_classes=no_classes)

y_val_new = [labels.index(i) for i in y_val]
y_val = tf.keras.utils.to_categorical(y_val_new, num_classes=no_classes)

In [16]:
# Cell 10: Define the residual block for the ResNet
def residual_block(x, filters, kernel_size=3, stride=1, use_bias=True, shortcut=True):
    res = x
    if stride != 1 or x.shape[-1] != filters:
        res = layers.Conv2D(filters, kernel_size=1, strides=stride, use_bias=use_bias)(res)
        res = layers.BatchNormalization()(res)

    x = layers.Conv2D(filters, kernel_size=kernel_size, strides=stride, padding='same', use_bias=use_bias)(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)

    x = layers.Conv2D(filters, kernel_size=kernel_size, strides=1, padding='same', use_bias=use_bias)(x)
    x = layers.BatchNormalization()(x)

    if shortcut:
        x = layers.add([x, res])
    
    x = layers.ReLU()(x)
    return x

In [17]:
# Cell 11: Build the ResNet model
def build_resnet(input_shape=(224, 224, 3), num_classes=no_classes):
    inputs = layers.Input(shape=input_shape)
    x = layers.Conv2D(64, kernel_size=7, strides=2, padding='same')(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    x = layers.MaxPooling2D(pool_size=3, strides=2, padding='same')(x)

    # Conv2_x
    for _ in range(3):
        x = residual_block(x, filters=64)

    # Conv3_x
    x = residual_block(x, filters=128, stride=2)
    for _ in range(3):
        x = residual_block(x, filters=128)

    # Conv4_x
    x = residual_block(x, filters=256, stride=2)
    for _ in range(5):
        x = residual_block(x, filters=256)

    # Conv5_x
    x = residual_block(x, filters=512, stride=2)
    for _ in range(2):
        x = residual_block(x, filters=512)

    x = layers.GlobalAveragePooling2D()(x)
    outputs = layers.Dense(num_classes, activation='softmax')(x)

    model = models.Model(inputs=inputs, outputs=outputs)
    return model

In [18]:
# Cell 12: Instantiate and compile the model
resnet_model = build_resnet(input_shape=(224, 224, 3), num_classes=no_classes)

resnet_model.compile(optimizer='adam', 
                     loss='categorical_crossentropy', 
                     metrics=['accuracy', 
                              tf.keras.metrics.Precision(), 
                              tf.keras.metrics.Recall(), 
                              tf.keras.metrics.AUC(), 
                              tf.keras.metrics.TruePositives(), 
                              tf.keras.metrics.TrueNegatives(), 
                              tf.keras.metrics.FalsePositives(), 
                              tf.keras.metrics.FalseNegatives()])

In [21]:
# Cell 14: Define callbacks (with early stopping)
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.001)
checkpoint = ModelCheckpoint('best_model.keras', monitor='val_loss', save_best_only=True)


In [22]:
# Cell 15: Data augmentation (to prevent overfitting)
datagen = ImageDataGenerator(rotation_range=20, width_shift_range=0.2, height_shift_range=0.2, 
                             horizontal_flip=True, fill_mode='nearest')

In [23]:
# Train the model with callbacks
history = resnet_model.fit(
    datagen.flow(X_train, y_train, batch_size=32),
    validation_data=(X_val, y_val),
    epochs=no_epochs,
    callbacks=[early_stopping, reduce_lr, checkpoint],
    verbose=1
)


Epoch 1/100


/opt/conda/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
I0000 00:00:1729702361.314795     127 service.cc:145] XLA service 0x798250001850 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1729702361.314867     127 service.cc:153]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
I0000 00:00:1729702361.314875     127 service.cc:153]   StreamExecutor device (1): Tesla T4, Compute Capability 7.5
I0000 00:00:1729702377.122379     127 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


149/149 ━━━━━━━━━━━━━━━━━━━━ 121s 542ms/step - accuracy: 0.0981 - auc: 0.6804 - false_negatives: 2384.1667 - false_positives: 110.1133 - loss: 3.8560 - precision: 0.1550 - recall: 0.0092 - true_negatives: 94049.4062 - true_positives: 30.1800 - val_accuracy: 0.0353 - val_auc: 0.5016 - val_false_negatives: 1188.0000 - val_false_positives: 68.0000 - val_loss: 10.4430 - val_precision: 0.0145 - val_recall: 8.4104e-04 - val_true_negatives: 46303.0000 - val_true_positives: 1.0000 - learning_rate: 0.0010
Epoch 2/100
149/149 ━━━━━━━━━━━━━━━━━━━━ 64s 410ms/step - accuracy: 0.2401 - auc: 0.8772 - false_negatives: 2216.4133 - false_positives: 197.2067 - loss: 2.6218 - precision: 0.4743 - recall: 0.0747 - true_negatives: 93771.9922 - true_positives: 193.0533 - val_accuracy: 0.0723 - val_auc: 0.6693 - val_false_negatives: 1150.0000 - val_false_positives: 330.0000 - val_loss: 4.5198 - val_precision: 0.1057 - val_recall: 0.0328 - val_true_negatives: 46041.0000 - val_true_positives: 39.0000 - learning_

In [31]:
import os
import cv2
import numpy as np

def load_and_preprocess_image(image_path, target_size=(224, 224)):
    img = cv2.imread(image_path)  # Load the image
    img = cv2.resize(img, target_size)  # Resize to target size
    img = img.astype('float32') / 255.0  # Normalize the image
    img = np.expand_dims(img, axis=0)  # Add batch dimension
    return img

# Specify the path to the dataset folder
dataset_path = '/kaggle/input/indian-medicinal-leaves-dataset/Indian Medicinal Leaves Image Datasets/Medicinal plant dataset'

# Test with an image from the dataset
for label in labels:
    folder_path = os.path.join(dataset_path, label)
    img_list = os.listdir(folder_path)
    
    # Get a sample image from each class
    for img_name in img_list[:1]:  # Just taking the first image from each class for testing
        image_path = os.path.join(folder_path, img_name)
        new_image = load_and_preprocess_image(image_path)

        # Predict the class
        predictions = resnet_model.predict(new_image)
        predicted_class_index = np.argmax(predictions, axis=1)

        # Get the corresponding label
        predicted_label = labels[predicted_class_index[0]]
        print(f'Image: {img_name} | True label: {label} | Predicted class: {predicted_label}')


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
Image: 2863.jpg | True label: Nooni | Predicted class: Nooni
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
Image: 2907.jpg | True label: Nithyapushpa | Predicted class: Nithyapushpa
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
Image: 3750.jpg | True label: Basale | Predicted class: Basale
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
Image: 3014.jpg | True label: Pomegranate | Predicted class: Pomegranate
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
Image: 3138.jpg | True label: Honge | Predicted class: Honge
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
Image: 5584.jpg | True label: Lemon_grass | Predicted class: Aloevera
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
Image: 4009.jpg | True label: Mint | Predicted class: Mint
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
Image: 1700.jpg | True label: Betel_Nut | Predicted class: Betel_Nut
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
Image: 2375.jpg | True label: Nagadali | Predicted class: Nagadali
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
Image: 3863.jpg | True

In [39]:
import cv2
import numpy as np

def load_and_preprocess_image(image_path, target_size=(224, 224)):
    img = cv2.imread(image_path)  # Load the image
    img = cv2.resize(img, target_size)  # Resize to target size
    img = img.astype('float32') / 255.0  # Normalize the image
    img = np.expand_dims(img, axis=0)  # Add batch dimension
    return img

# Specify the path to your image
image_path = '/kaggle/input/indian-medicinal-leaves-dataset/Indian Medicinal Leaves Image Datasets/Medicinal plant dataset/Avacado/1002.jpg'  # Change this to the path of the image you want to test

# Preprocess the image
new_image = load_and_preprocess_image(image_path)

# Make a prediction
predictions = resnet_model.predict(new_image)
predicted_class_index = np.argmax(predictions, axis=1)

# Get the corresponding label
predicted_label = labels[predicted_class_index[0]]

print(f'Predicted class: {predicted_label}')


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
Predicted class: Avacado


In [41]:
# Save the model in HDF5 format
resnet_model.save('resnet_mod.h5')
